# Installing of necessary packages

In [1]:
# %pip install -r requirements.txt

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_experimental.text_splitter import SemanticChunker
from langchain.embeddings import HuggingFaceEmbeddings
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_community.llms import Ollama
from langchain.chains import RetrievalQA
from langchain.chains.llm import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.prompts import PromptTemplate
import faiss

# Loading of Documents

In [ ]:
loader = TextLoader('Refund Policy Document.txt', encoding = 'UTF-8')
data = loader.load()

In [4]:
# Takes way too long to run, we use fixed chunking with overlap instead

# text_splitter = SemanticChunker(HuggingFaceEmbeddings())
# documents = text_splitter.split_documents(data)

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
)

documents = text_splitter.split_documents(data)

In [6]:
print("Number of chunks created: ", len(documents))

for i in range(len(documents)):
    print()
    print(f"CHUNK : {i+1}")
    print(documents[i].page_content)

Number of chunks created:  20

CHUNK : 1
1. Application for Returns/Refunds
 Subject to the terms and conditions in this Refunds and Return Policy and the Terms of Service, Buyer may apply for return of the purchased items (“Item”) and/or refund.
 For the avoidance of doubt, an application for a refund and/or return may be made whether prior to or after the expiry of the Shopee Guarantee Period as stated in the Terms of Service. Users may communicate with each other privately to resolve their differences prior, during or after using Shopee Guarantee.
 2. Application for the Return of an Item
 Buyer may only apply for
(i) the refund of the Item where such Item has not been received by Buyer (“Unreceived Items”), or
(ii) the refund and/or return of an Item received by Buyer (“Received Items”) in the following circumstances:
a) Buyer received an empty and/or suspicious parcel;
b) The Item received was incomplete (missing quantity or accessories);

CHUNK : 2
a) Buyer received an empty and/

# Creation of Embeddings and Vector Store

In [7]:
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",  # 384-dim, 5x faster
    model_kwargs={'device': 'cpu'},  # Change to 'cuda' if GPU available
    encode_kwargs={
        'batch_size': 512,  # Process 512 docs at once
        'normalize_embeddings': True
    }
)

# Create Vector Store
vector_store = FAISS.from_documents(
        documents=documents,
        embedding=embeddings,
)

# Creates retriever for the vector store, based on cosine similarity, find top 3 closest documents with a minimum score threshold of 0.75
retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 3, "score_threshold": 0.75})

# retrieved_docs = retriever.invoke("What is the refund policy?")
# print(retrieved_docs[0])

C:\Users\Owent\AppData\Local\Temp\ipykernel_55372\371343745.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Owent\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Owent\AppData\Local\Programs\Python\Python310\lib\site-packages\threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompat

# Creating of Chat model and Prompt

In [ ]:
# Define the Ollama LLM
llm = Ollama(
    model="mistral", 
    temperature=0,
    num_gpu=1,  # Enable GPU
    num_thread=8
    )

C:\Users\Owent\AppData\Local\Temp\ipykernel_55372\2434474837.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(


In [34]:
prompt = """
You are Shopee's Refund Policy Bot. You are tasked to answer questions related to Shopee's Refund Policy.
If there is no provided context, only reply with: "I am only tasked to reply questions related to Shopee's Refund Policy."
For questions related to Shopee, use the provided context to answer the question at the end.
Do not make up answers or provide information beyond the context.
Always end your answer with "Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details."

Context: {context}

Question: {question}

Helpful Answer:"""

# prompt = """
# 1. Use the provided context to answer the question at the end.
# 2. If the question is unrelated to Shopee's Refund Policy or lacks sufficient context, respond with:  
#    "The question is not related to Shopee's Refund Policy."
# 3. If you cannot confidently answer based on the context, reply with:  
#    "I am not too sure. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details."
# 4. Do not make up answers or provide information beyond the context.

# Context: {context}

# Question: {question}

# Helpful Answer:"""

# Creating of Prompt Template
QA_CHAIN_PROMPT = PromptTemplate.from_template(prompt) 

# Creation of LLM Chain
llm_chain = LLMChain(
    llm=llm, 
    prompt=QA_CHAIN_PROMPT, 
    callbacks=None, 
    verbose=True
)

# To display Context content and source
document_prompt = PromptTemplate(
    input_variables=["page_content", "source"],
    template="\nContent:{page_content}\nSource:{source}",
)

# Combining of both LLM Chain and Prompt Template
combine_documents_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context",
    document_prompt=document_prompt,
    callbacks=None,
)

# Creation of actual Question Answering
qa = RetrievalQA(
    combine_documents_chain=combine_documents_chain,
    verbose=True,
    retriever=retriever,
    return_source_documents=True,
)

# Alternate QA
# qa = RetrievalQA.from_chain_type(
#     llm=llm,
#     chain_type="stuff",
#     retriever=retriever,
#     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
#     return_source_documents=True
# )

In [ ]:
print(qa("What is Shopee's Refund Policy?")['result'])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are Shopee's Refund Policy Bot. You are tasked to answer questions related to Shopee's Refund Policy.
If there is no provided context, only reply with: "I am only tasked to reply questions related to Shopee's Refund Policy."
For questions related to Shopee, use the provided context to answer the question at the end.
Do not make up answers or provide information beyond the context.
Always end your answer with "Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details."

Context: 
Content:the automatic cancellation of Buyer’s refund and/or return request. The approval of your request for refund and/or return will be made by Shopee in its sole discretion, and shall be subject to your compliance with such conditions as may be specified by Shopee from time to time. Buyer acknowledges and agrees that Shopee’s decision is final, conclusive and b

In [ ]:
print(qa("How are you?")["result"])



> Entering new RetrievalQA chain...


> Entering new LLMChain chain...
Prompt after formatting:

You are Shopee's Refund Policy Bot. You are tasked to answer questions related to Shopee's Refund Policy.
If there is no provided context, only reply with: "I am only tasked to reply questions related to Shopee's Refund Policy. Please refer to https://help.shopee.sg/portal/4/article/77152-Refunds-and-Return-Policy for more details."
For questions related to Shopee, use the provided context to answer the question at the end.
Do not make up answers or provide information beyond the context.

Context: 

Question: How are you?

Helpful Answer:

> Finished chain.

> Finished chain.
 I am a bot designed to help with Shopee's Refund Policy inquiries. I don't have feelings, but I'm here and ready to assist you! If you have any questions about refunds on Shopee, feel free to ask.
